This file does some exploratory data analysis on the garbage dataset

In [ ]:
# import pandas
import pandas as pd

# read in training dataset and print to see if structure is correct:
train_df = pd.read_csv("../Data/one-indexed-files-notrash_train.txt", sep=" ", header=None, names=["filename", "label"])
print(train_df.head())

           filename  label
0  cardboard202.jpg      3
1      paper472.jpg      2
2      paper522.jpg      2
3      glass189.jpg      1
4      glass325.jpg      1


In [4]:
# read in validation and training set:
val_df = pd.read_csv("../Data/one-indexed-files-notrash_val.txt", sep=" ", header=None, names=["filename", "label"])
test_df = pd.read_csv("../Data/one-indexed-files-notrash_test.txt", sep=" ", header=None, names=["filename", "label"])